In [27]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [28]:
# Set up the training, validation, and testing data directories
train_dir = r"C:\Users\hp\Desktop\ResNet 101\train"
val_dir = r"C:\Users\hp\Desktop\ResNet 101\val"
test_dir = r"C:\Users\hp\Desktop\ResNet 101\test"

In [29]:
# Define the input image size
img_size = 224

# Define the number of classes
num_classes = 14

In [31]:
from keras.preprocessing.image import ImageDataGenerator

# Define data augmentation parameters for the training dataset
train_datagen = ImageDataGenerator(
        rescale=1./255,  # rescale pixel values to [0,1]
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        shear_range=0.2,  # randomly apply shearing transformations
        zoom_range=0.2,  # randomly zoom in and out
        horizontal_flip=True)  # randomly flip images horizontally

# Define data augmentation parameters for the validation dataset
val_datagen = ImageDataGenerator(rescale=1./255)  # just rescale pixel values

# Load and prepare your training data
train_data = train_datagen.flow_from_directory(
        r"C:\Users\hp\Desktop\ResNet 101\train",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

# Load and prepare your validation data
val_data = val_datagen.flow_from_directory(
        r"C:\Users\hp\Desktop\ResNet 101\val",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 698 images belonging to 14 classes.
Found 140 images belonging to 14 classes.


In [32]:
# Set up the data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical')

Found 698 images belonging to 14 classes.
Found 140 images belonging to 14 classes.


In [34]:
# Load the ResNet101 model and remove the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

C:\Users\hp\AppData\Roaming\Python\Python36\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of a268eb855778b3df3c7506639542a6af so we will re-download the data.
94658560/94653016 [==============================] - 11s 0us/step


In [35]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [36]:
# Add new layers for classification
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dense(524, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)

In [37]:
# Create the model
model = Model(inputs=base_model.input, outputs=x)

In [56]:
from tensorflow.keras.optimizers import Adam

In [57]:
# Create an instance of the Adam optimizer
adam_optimizer = Adam()

In [58]:
# Compile the model
model.compile(optimizer=adam_optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [61]:
# Compile the model
model.compile(optimizer=Adam, loss='categorical_crossentropy', metrics=['accuracy'])

ValueError: ('Could not interpret optimizer identifier:', <class 'tensorflow.python.keras.optimizers.Adagrad'>)

In [54]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.n // val_generator.batch_size)

Epoch 1/10
22/22 [==============================] - 133s 6s/step - loss: 14.9958 - acc: 0.0702 - val_loss: 14.9428 - val_acc: 0.0714
Epoch 2/10
22/22 [==============================] - 125s 6s/step - loss: 14.9857 - acc: 0.0702 - val_loss: 14.7749 - val_acc: 0.0714
Epoch 3/10
22/22 [==============================] - 124s 6s/step - loss: 14.9807 - acc: 0.0702 - val_loss: 15.1107 - val_acc: 0.0714
Epoch 4/10
22/22 [==============================] - 127s 6s/step - loss: 14.9857 - acc: 0.0702 - val_loss: 14.7749 - val_acc: 0.0714
Epoch 5/10
22/22 [==============================] - 153s 7s/step - loss: 14.9958 - acc: 0.0702 - val_loss: 14.9428 - val_acc: 0.0714
Epoch 6/10
22/22 [==============================] - 142s 6s/step - loss: 14.9807 - acc: 0.0702 - val_loss: 15.1107 - val_acc: 0.0714
Epoch 7/10
22/22 [==============================] - 127s 6s/step - loss: 14.9807 - acc: 0.0702 - val_loss: 14.9428 - val_acc: 0.0714
Epoch 8/10
22/22 [==============================] - 139s 6s/step - lo

In [47]:
# Evaluate the model on the testing set
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical')

Found 147 images belonging to 14 classes.


In [48]:
test_loss, test_acc = model.evaluate(test_generator)

5/5 [==============================] - 12s 2s/step - loss: 14.8403 - acc: 0.0816


In [49]:
print('Test accuracy:', test_acc)

Test accuracy: 0.08163265
